In [1]:
import pandas as pd
import numpy as np

In [ ]:
rename_map = {"Country name": "Country",
              "country": "Country",
              "Ladder score": "Happiness score",
              "upperwhisker": "Whisker-high",
              "lowerwhisker": "Whisker-low"
}

keep = ["Country", "Regional indicator", "Year", "Happiness score", "Whisker-high", "Whisker-low", "Explained by: GDP per capita",
        "Explained by: Log GDP per capita",  "Explained by: Social support", "Explained by: Health life expectancy",
        "Explained by: Freedom to make life choices", "Explained by: Generosity", "Explained by: Perceptions of corruption"]

dfs = {}
for i in range(15, 25):
    dfs[i] = pd.read_csv(f"old_data/WHR{i}.csv")

for i in dfs.keys():
    dfs[i]["Year"] = pd.to_datetime(2000+i, format="%Y") #2000+i
    dfs[i].rename(rename_map, axis=1, inplace=True)    
    dfs[i] = dfs[i][[x for x in keep if x in dfs[i].columns]]

df = pd.concat(dfs.values())

## DATA CLEANING ##

rename_countries1 = {}
[rename_countries1.update({i: i[:-1]}) for i in sorted(df["Country"].value_counts().index.values) if "*" in i];

rename_countries2 = { ## found by manual analysis of the data
    "Czech Republic": "Czechia",
    "Hong Kong S.A.R. of China": "Hong Kong",
    "Hong Kong SAR, China": "Hong Kong",
    "North Cyprus": "Northern Cyprus",
    "Palestinian Territories": "State of Palestine", ## used in 2025
    "Taiwan Province of China": "Taiwan",
    "Trinidad & Tobago": "Trinidad and Tobago",
    "Turkey": "Turkiye",
    "Eswatini": "Swaziland/Eswatini", ## not fully sure what to do with these three
    "Eswatini, Kingdom of": "Swaziland/Eswatini", 
    "Swaziland": "Swaziland/Eswatini"
}

df = df[df["Country"] != "xx"] ## remove empty row
df["Country"] = df["Country"].replace(rename_countries1)
df["Country"] = df["Country"].replace(rename_countries2)
df.head(3)

,Country,Regional indicator,Year,Happiness score,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Whisker-high,Whisker-low,Explained by: GDP per capita
0,Switzerland,Western Europe,2015-01-01,7.587,1.396505,1.349505,0.665573,0.296775,0.419777,NaN,NaN,NaN
1,Iceland,Western Europe,2015-01-01,7.561,1.302324,1.402231,0.628772,0.436297,0.141451,NaN,NaN,NaN
2,Denmark,Western Europe,2015-01-01,7.527,1.325478,1.360581,0.649384,0.341386,0.483573,NaN,NaN,NaN


---

In [ ]:
df_global = pd.read_csv("old_data/Global Economy Indicators.csv")
df_global.columns = df_global.columns.str.strip()
for i in df_global.columns:
    if df_global[i].dtype == object:
        df_global[i] = df_global[i].str.strip()
df_global["Year"] = pd.to_datetime(df_global["Year"], format="%Y")

df2 = df_global[df_global["Year"] >= pd.to_datetime(2015, format="%Y")]

In [7]:
full_df = pd.merge(left=df, right=df2, how="left", left_on=['Country', 'Year'], right_on=['Country', 'Year'])

In [8]:
full_df.sort_values(by="Country").head(3)

,Country,Regional indicator,Year,Happiness score,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Whisker-high,...,Household consumption expenditure (including Non-profit institutions serving households),Imports of goods and services,Manufacturing (ISIC D),"Mining, Manufacturing, Utilities (ISIC C-E)",Other Activities (ISIC J-P),Total Value Added,"Transport, storage and communication (ISIC I)","Wholesale, retail trade, restaurants and hotels (ISIC G-H)",Gross National Income(GNI) in USD,Gross Domestic Product (GDP)
1509,Afghanistan,NaN,2024-01-01,1.7210,0.628000,0.000000,0.000000,0.091000,0.088000,1.77500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
934,Afghanistan,South Asia,2020-01-01,2.5669,0.300706,0.356434,0.000000,0.135235,0.001226,2.62827,...,1.899194e+10,6.983058e+09,1.546263e+09,2.206409e+09,7.098671e+09,1.924517e+10,1.929933e+09,1.987511e+09,2.033238e+10,2.014344e+10
311,Afghanistan,NaN,2016-01-01,3.3600,NaN,0.110368,0.164305,0.312680,0.071116,3.43200,...,1.444640e+10,5.841752e+09,7.413601e+08,1.296204e+09,7.073713e+09,1.716008e+10,1.583227e+09,1.978781e+09,1.819913e+10,1.801956e+10


In [9]:
rename_cols = {
    "Happiness score": "Happiness_score",
    "Gross Domestic Product (GDP)": "GDP",
}
full_df.rename(rename_cols, axis=1, inplace=True)
subdf = full_df[full_df["Year"] < pd.to_datetime(2022, format="%Y")]
subdf["GDP"].dropna()

0       7.020000e+11
1       1.751721e+10
2       3.030000e+11
3       3.860000e+11
4       1.560000e+12
            ...     
1079    2.373416e+09
1080    1.761486e+10
1081    1.106992e+10
1082    2.411815e+10
1083    1.493901e+10
Name: GDP, Length: 957, dtype: float64

In [ ]:
import statsmodels.api as stats

model = stats.formula.ols("Happiness_score ~ GDP", data = full_df, missing="drop").fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Happiness_score   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     33.56
Date:                Mon, 15 Dec 2025   Prob (F-statistic):           9.38e-09
Time:                        16:20:39   Log-Likelihood:                -1471.9
No. Observations:                 957   AIC:                             2948.
Df Residuals:                     955   BIC:                             2957.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.3996      0.038    143.364      0.000       5.326       5.473
GDP         9.703e-14   1.68e-14      5.793      0.000    6.42e-14     1.3e-13
==============================================================================
Omnibus:                       65.024   Durbin-Watson:                   0.188
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.214
Skew:                          -0.013   Prob(JB):                     9.10e-06
Kurtosis:                       2.237   Cond. No.                     2.32e+12
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.32e+12. This might indicate that there are
strong multicollinearity or other numerical problems.
"""